In [5]:
import humanfriendly

def d(n):
    return humanfriendly.format_size(n, binary=True)

# Running benchmarks for vcztools

Compare time to get POS field from BCF
```
jk@holly$ time bcftools query -f '%POS\n' chr21_10_5.bcf > tmp/pos2.txt

real    8m57.929s
user    8m53.253s
sys     0m4.230s
```

```
jk@holly$ time vcztools query -f '%POS\n' chr21_10_5.zarr > tmp/pos.txt

real    0m51.453s
user    0m51.809s
sys     0m2.781s
```

Check they're identical:
```
jk@holly$ diff tmp/pos2.txt tmp/pos.txt
```

How many variants do we have?
```
jk@holly$ wc -l tmp/pos.txt
2365367 tmp/pos.txt
```

Get the position of the last n - 10000 variant:
```

$ tail -n 10000 tmp/pos.txt | head -n 1
47958263
```

```
$ time bcftools view chr21_10_5.bcf -r1:47958263- | wc -c
4001339854

real    0m24.998s
user    0m23.388s
sys     0m2.814s
```

```
jk@holly$ time bcftools view chr21_10_5.vcf.gz -r1:47958263- | wc -c
4001339730

real    1m2.770s
user    1m0.978s
sys     0m2.580s
```

In [3]:
d(4001339854 / 25)

'152.64 MiB'

In [7]:
d(4001339730 / 63)

'60.57 MiB'

We get a data rate of 152 MiB/s from BCF vs 60 from VCF

```
$ time vcztools view chr21_10_5.zarr -r1:47958263- | wc -c
4001339815

real    0m30.685s
user    0m33.794s
sys     0m7.438s
```


In [10]:
d(4001339815 / 30.7)

'124.3 MiB'

In [8]:
d(4001339730 / 43.56)

'87.6 MiB'

It's useful to know what the rate at which VCF can be parsed is, so we wrote a simple program that does this using htslib. On the subset of VCF that we have here, we get

```
$ time ../software/vcf_parse/vcf_parse data/tmp/subset.vcf 
Reading VCF data/tmp/subset.vcf
read = 10000 records

real    0m31.015s
user    0m30.281s
sys     0m0.724s
```
Size is
```
$ ls data/tmp/subset.vcf -l
-rw-r--r-- 1 jk jk 4001339730 Jan 23 12:45 data/tmp/subset.vcf
```


In [11]:
d(4001339730 / 31)

'123.1 MiB'

As it happens, this is almost identical to the rate output by vcztools here.

In [13]:
d(4001339730)

'3.73 GiB'